## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yima,CN,34.7381,111.8839,99.68,21,17,20.11,few clouds
1,1,Gorontalo,ID,0.5412,123.0595,79.14,90,100,1.79,overcast clouds
2,2,Qaanaaq,GL,77.4840,-69.3632,33.55,79,100,6.53,overcast clouds
3,3,Puerto Ayora,EC,-0.7393,-90.3518,69.55,96,80,4.00,broken clouds
4,4,Dikson,RU,73.5069,80.5464,35.62,79,97,11.92,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Gorontalo,ID,0.5412,123.0595,79.14,90,100,1.79,overcast clouds
5,5,Butaritari,KI,3.0707,172.7902,81.28,79,6,6.64,light rain
9,9,Vaini,TO,-21.2000,-175.2000,76.62,88,69,19.71,broken clouds
10,10,Celestun,MX,20.8667,-90.4000,77.14,85,100,10.02,overcast clouds
11,11,Atuona,PF,-9.8000,-139.0333,77.85,75,11,20.07,few clouds
19,19,Kupang,ID,-10.1667,123.5833,85.23,57,32,9.86,scattered clouds
28,28,Tara,RU,56.8944,74.3710,77.02,50,19,3.18,few clouds
29,29,Marovoay,MG,-16.1000,46.6333,82.81,44,6,8.32,clear sky
30,30,Aojiang,CN,22.9421,116.0359,85.30,71,100,15.90,moderate rain
33,33,Ranau,MY,5.9538,116.6641,78.01,88,44,1.05,scattered clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                232
City                   232
Country                232
Lat                    232
Lng                    232
Max Temp               232
Humidity               232
Cloudiness             232
Wind Speed             232
Current Description    232
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='any')

In [63]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Gorontalo,ID,79.14,overcast clouds,0.5412,123.0595,
5,Butaritari,KI,81.28,light rain,3.0707,172.7902,
9,Vaini,TO,76.62,broken clouds,-21.2000,-175.2000,
10,Celestun,MX,77.14,overcast clouds,20.8667,-90.4000,
11,Atuona,PF,77.85,few clouds,-9.8000,-139.0333,
19,Kupang,ID,85.23,scattered clouds,-10.1667,123.5833,
28,Tara,RU,77.02,few clouds,56.8944,74.3710,
29,Marovoay,MG,82.81,clear sky,-16.1000,46.6333,
30,Aojiang,CN,85.30,moderate rain,22.9421,116.0359,
33,Ranau,MY,78.01,scattered clouds,5.9538,116.6641,


In [64]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()       
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [80]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216 entries, 1 to 695
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 216 non-null    object 
 1   Country              216 non-null    object 
 2   Max Temp             216 non-null    float64
 3   Current Description  216 non-null    object 
 4   Lat                  216 non-null    float64
 5   Lng                  216 non-null    float64
 6   Hotel Name           216 non-null    object 
dtypes: float64(3), object(4)
memory usage: 13.5+ KB


In [81]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [91]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [92]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))